In [ ]:
import huggingface_hub as hf
import pandas as pd

In [ ]:
test_filenames = [
  'ALL/test-00000-of-00001.parquet',
]  
train_filenames = [
  'ALL/train-00001-of-00009.parquet',
  'ALL/train-00002-of-00009.parquet',
  'ALL/train-00003-of-00009.parquet',
  'ALL/train-00004-of-00009.parquet',
  'ALL/train-00005-of-00009.parquet',
  'ALL/train-00006-of-00009.parquet',
  'ALL/train-00007-of-00009.parquet',
  'ALL/train-00008-of-00009.parquet',
]

In [ ]:
train_df = pd.DataFrame([])
test_df = pd.DataFrame([])

for filename in train_filenames:
  path = hf.hf_hub_download(
    repo_id='BAAI/TACO',
    repo_type='dataset',
    filename=filename,
  )

  if train_df.empty:
    train_df = pd.read_parquet(path=path)
  else:
    train_df = pd.concat([train_df, pd.read_parquet(path)], axis=1)

for filename in test_filenames:
  path = hf.hf_hub_download(
    repo_id='BAAI/TACO',
    repo_type='dataset',
    filename=filename,
  )

  if test_df.empty:
    test_df = pd.read_parquet(path)
  else:
    test_df = pd.concat([test_df, pd.read_parquet(path)], axis=1)

train-00001-of-00009.parquet:  10%|9         | 31.5M/327M [00:00<?, ?B/s]